In [134]:
# Base utilities
import os

# Data Mining
import math
import random
random_state = 42
random.seed(random_state)
seed=random_state
import numpy as np 
import pandas as pd
import geopandas as gpd
import osmnx as ox
import pandana as pdn
import pickle as pkl

# Plot
import matplotlib.pyplot as plt
import matplotlib.pyplot as plot
import seaborn as sns

# Learning
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, train_test_split, KFold

# Models
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Directories
for d in ["data", "logs", "models", "networks"]:
    if not os.path.isdir(d):
        os.mkdir(d)

# Pylogit
import collections.abc
collections.Iterable = collections.abc.Iterable
import pylogit as pl                   # For MNL model estimation and

from collections import OrderedDict    # For recording the model specification 


## Trips

In [148]:
# Bring the trips
trips = pd.read_csv(f"data/Gipuzkoa_MT2.csv")

In [149]:
trips.columns

Index(['Unnamed: 0', 'Prov_Ori', 'Prov_Des', 'Com_Ori', 'Com_Des', 'Mun_Ori',
       'Mun_Des', 'Act_Ori', 'Act_Des', 'Proposito', 'Modo', 'Hora_Ini',
       'Dur_Tot', 'Recur', 'Bill', 'Park', 'Peaje', 'Provincia', 'Municipio',
       'Comarca', 'Per_hog', 'Turismos', 'Motos', 'Sexo', 'Edad', 'Actividad',
       'crnt_tur', 'crnt_mot', 'crnt_otr', 'O_long', 'O_lat', 'D_long',
       'D_lat'],
      dtype='object')

In [136]:
# Changes primary modes to a different model: auto or green_modes

trips = trips[trips['Modo'] != 'Otros']

# 5 MODOS

# trips.loc[trips.Modo=="Coche pasajero","Modo"] = "Coche"
# trips.loc[trips.Modo=="Coche conductor","Modo"] = "Coche"
# trips.loc[trips.Modo=="Moto","Modo"] = "Coche"
# trips.loc[trips.Modo=="Taxi","Modo"] = "Coche"
# trips.loc[trips.Modo=="Autobús interurbano","Modo"] = "Autobus"
# trips.loc[trips.Modo=="Autobús urbano","Modo"] = "Autobus"
# trips.loc[trips.Modo=="Autobús empresa - colegio","Modo"] = "Autobus"
# trips.loc[trips.Modo=="EUSKOTREN","Modo"] = "Tren"
# trips.loc[trips.Modo=="RENFE","Modo"] = "Tren"

# 2 MODOS, coche y green

# trips.loc[trips.Modo=="Coche pasajero","Modo"] = "Coche"
# trips.loc[trips.Modo=="Coche conductor","Modo"] = "Coche"
# trips.loc[trips.Modo=="Moto","Modo"] = "Coche"
# trips.loc[trips.Modo=="Taxi","Modo"] = "Coche"
# trips.loc[trips.Modo=="Autobús interurbano","Modo"] = "Green"
# trips.loc[trips.Modo=="Autobús urbano","Modo"] = "Green"
# trips.loc[trips.Modo=="Autobús empresa - colegio","Modo"] = "Green"
# trips.loc[trips.Modo=="EUSKOTREN","Modo"] = "Green"
# trips.loc[trips.Modo=="RENFE","Modo"] = "Green"
# trips.loc[trips.Modo=="Andando","Modo"] = "Green"
# trips.loc[trips.Modo=="Bicicleta","Modo"] = "Green"

# 2 MODOS, TP y coche

trips.loc[trips.Modo=="Coche pasajero","Modo"] = "Coche"
trips.loc[trips.Modo=="Coche conductor","Modo"] = "Coche"
trips.loc[trips.Modo=="Moto","Modo"] = "Coche"
trips.loc[trips.Modo=="Taxi","Modo"] = "Coche"
trips.loc[trips.Modo=="Autobús interurbano","Modo"] = "TP"
trips.loc[trips.Modo=="Autobús urbano","Modo"] = "TP"
trips.loc[trips.Modo=="Autobús empresa - colegio","Modo"] = "TP"
trips.loc[trips.Modo=="EUSKOTREN","Modo"] = "TP"
trips.loc[trips.Modo=="RENFE","Modo"] = "TP"
trips = trips[trips['Modo'] != 'Andando']
trips = trips[trips['Modo'] != 'Bicicleta']

# 3 MODOS

# trips.loc[trips.Modo=="Coche pasajero","Modo"] = "Coche"
# trips.loc[trips.Modo=="Coche conductor","Modo"] = "Coche"
# trips.loc[trips.Modo=="Moto","Modo"] = "Coche"
# trips.loc[trips.Modo=="Taxi","Modo"] = "Coche"
# trips.loc[trips.Modo=="Autobús interurbano","Modo"] = "PT"
# trips.loc[trips.Modo=="Autobús urbano","Modo"] = "PT"
# trips.loc[trips.Modo=="Autobús empresa - colegio","Modo"] = "PT"
# trips.loc[trips.Modo=="EUSKOTREN","Modo"] = "PT"
# trips.loc[trips.Modo=="RENFE","Modo"] = "PT"
# trips.loc[trips.Modo=="Andando","Modo"] = "Active modes"
# trips.loc[trips.Modo=="Bicicleta","Modo"] = "Active modes"

# 4 MODOS

# trips.loc[trips.Modo=="Coche pasajero","Modo"] = "Coche"
# trips.loc[trips.Modo=="Coche conductor","Modo"] = "Coche"
# trips.loc[trips.Modo=="Moto","Modo"] = "Coche"
# trips.loc[trips.Modo=="Taxi","Modo"] = "Coche"
# trips.loc[trips.Modo=="Autobús interurbano","Modo"] = "PT"
# trips.loc[trips.Modo=="Autobús urbano","Modo"] = "PT"
# trips.loc[trips.Modo=="Autobús empresa - colegio","Modo"] = "PT"
# trips.loc[trips.Modo=="EUSKOTREN","Modo"] = "PT"
# trips.loc[trips.Modo=="RENFE","Modo"] = "PT"
# trips.loc[trips.Modo=="Andando","Modo"] = "Andando"
# trips.loc[trips.Modo=="Bicicleta","Modo"] = "Bicicleta"

# Vuelvo a mostrar los modos de transporte

trips=trips.reset_index(drop=True)

for p in [f"{label}: {trips[trips.Modo==label].shape[0]:,}" for label in trips.Modo.unique()]:
    print(p)
print(f"Total: {trips.shape[0]:,}")



Coche: 2,071
TP: 411
Total: 2,482


In [137]:
# Reducir el numero de trips en coche a la suma del resto de modos.

# Numero de green trips
condicion = trips['Modo'] != 'Coche'
green_trips = condicion.sum()

condicion = trips['Modo'] == 'Coche'
coche_trips = condicion.sum()
eliminar=coche_trips-green_trips

#eliminar=len(trips['MODO_INFORME']=='Coche')-green_trips

# Escoger aleatoriamente ese numero de trips de todos los del coche

# Filtrar las filas que cumplen con el modo de transporte "coche"
coche_rows = trips[trips['Modo'] == 'Coche']

# Seleccionar aleatoriamente X filas del conjunto de filas "coche_rows"
muestras_aleatorias = coche_rows.sample(n=eliminar)

# Eliminar las filas que no fueron seleccionadas aleatoriamente
trips = trips[~trips.index.isin(muestras_aleatorias.index)]

trips=trips.reset_index(drop=True)

# Vuelvo a mostrar los modos de transporte

for p in [f"{label}: {trips[trips.Modo==label].shape[0]:,}" for label in trips.Modo.unique()]:
    print(p)
print(f"Total: {trips.shape[0]:,}")

Coche: 411
TP: 411
Total: 822


In [147]:
p=trips[trips['Edad']==6]
# p=p[p['Modo']=='Coche']
p


,Unnamed: 0,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,...,Sexo,Edad,Actividad,crnt_tur,crnt_mot,crnt_otr,O_long,O_lat,D_long,D_lat
8,99,20,20,Urola Kosta,Debagoiena,Azkoitia,Bergara,Residencia Habitual,Trabajo habitual,Trabajo,...,1,6,1,1,2,2,-2.297017,43.156560,-2.415960,43.116299
31,312,20,20,Bidasoa,Bidasoa,Irun,Hondarribia,Trabajo habitual,Residencia Habitual,Trabajo,...,1,6,1,1,2,2,-1.818712,43.330582,-1.791522,43.366207
32,316,20,20,Bidasoa,Bidasoa,Irun,Hondarribia,Trabajo habitual,Residencia Habitual,Trabajo,...,2,6,1,1,2,2,-1.813104,43.328554,-1.802391,43.345133
33,318,20,20,Bidasoa,Bidasoa,Irun,Hondarribia,Trabajo habitual,Residencia Habitual,Trabajo,...,2,6,1,1,2,2,-1.809933,43.335561,-1.827935,43.341005
34,319,20,20,Bidasoa,Bidasoa,Hondarribia,Irun,Residencia Habitual,Trabajo habitual,Trabajo,...,2,6,1,1,2,2,-1.800407,43.372758,-1.761189,43.339585
44,405,20,20,Donostialdea,Bidasoa,Hernani,Irun,Residencia Habitual,Trabajo habitual,Trabajo,...,2,6,1,1,2,2,-1.962766,43.254109,-1.838306,43.327450
51,483,20,20,Donostialdea,Donostialdea,Hernani,Donostia/San Sebastian,Residencia Habitual,Trabajo habitual,Trabajo,...,1,6,1,1,1,2,-1.974963,43.265176,-1.972415,43.311541
52,485,20,20,Donostialdea,Donostialdea,Donostia/San Sebastian,Hernani,Trabajo habitual,Residencia Habitual,Trabajo,...,1,6,1,1,1,2,-1.963862,43.316341,-1.976907,43.274689
80,678,20,20,Goierri,Goierri,Lazkao,Beasain,Residencia Habitual,Trabajo habitual,Trabajo,...,2,6,1,1,2,2,-2.187684,43.038058,-2.218318,43.051002
86,703,20,20,Goierri,Goierri,Urretxu,Legazpi,Asuntos trabajo,Trabajo habitual,Trabajo,...,1,6,1,1,1,1,-2.322848,43.085342,-2.333940,43.065086


In [107]:
# Joven y mayor

def asignar_joven(row):
    if row['Edad'] in [2, 3, 4]:
        return 1
    else:
        return 0
def asignar_mayor(row):
    if row['Edad'] in [5, 6, 7]:
        return 1
    else:
        return 0

# Aplicar la función a cada fila y crear la nueva columna 'NuevaColumna'
trips['Joven'] = trips.apply(asignar_joven, axis=1)
trips['Mayor'] = trips.apply(asignar_mayor, axis=1)
trips = trips.drop(columns=['Edad'])
trips

,Unnamed: 0,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,...,Actividad,crnt_tur,crnt_mot,crnt_otr,O_long,O_lat,D_long,D_lat,Joven,Mayor
0,0,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,...,1,1,2,1,-1.968119,43.309597,-2.209558,43.043715,0,1
1,1,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,...,1,1,2,2,-2.507008,43.060521,-1.984274,43.317133,0,1
2,2,20,20,Donostialdea,Urola Kosta,Andoain,Zarautz,Trabajo habitual,Gestiones personales,Trabajo,...,1,1,2,2,-2.029870,43.202270,-2.175352,43.284143,0,1
3,5,20,20,Goierri,Donostialdea,Beasain,Andoain,Trabajo habitual,Residencia Habitual,Trabajo,...,1,1,2,2,-2.194873,43.047227,-2.023089,43.211462,0,1
4,6,20,20,Donostialdea,Goierri,Andoain,Beasain,Residencia Habitual,Trabajo habitual,Trabajo,...,1,1,2,2,-2.017355,43.221807,-2.199207,43.044323,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,5080,20,20,Bidasoa,Bidasoa,Irun,Hondarribia,Trabajo habitual,Residencia Habitual,Trabajo,...,1,1,1,2,-1.767785,43.336861,-1.827935,43.341005,0,1
2478,5081,20,20,Bidasoa,Bidasoa,Hondarribia,Irun,Centro estudios,Trabajo habitual,Trabajo,...,1,1,2,2,-1.818341,43.341179,-1.805404,43.333622,0,1
2479,5082,20,20,Bidasoa,Bidasoa,Irun,Hondarribia,Trabajo habitual,Centro estudios,Estudio,...,1,1,2,2,-1.826807,43.327675,-1.802130,43.372421,0,1
2480,5083,20,20,Bidasoa,Bidasoa,Hondarribia,Irun,Residencia Habitual,Trabajo habitual,Trabajo,...,1,1,2,2,-1.793067,43.369469,-1.849910,43.327468,0,1


In [109]:
trips= pd.concat([trips, pd.get_dummies(trips['Turismos'], prefix='Turismos')], axis=1)
trips = trips.drop(columns=['Turismos'])
trips['Turismos_1'] = trips['Turismos_1'].replace({True: 1, False: 0})
trips['Turismos_2'] = trips['Turismos_2'].replace({True: 1, False: 0})
trips['Turismos_3'] = trips['Turismos_3'].replace({True: 1, False: 0})
trips['Turismos_4'] = trips['Turismos_4'].replace({True: 1, False: 0})
trips

,Unnamed: 0,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,...,O_long,O_lat,D_long,D_lat,Joven,Mayor,Turismos_1,Turismos_2,Turismos_3,Turismos_4
0,0,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,...,-1.968119,43.309597,-2.209558,43.043715,0,1,0,0,1,0
1,1,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,...,-2.507008,43.060521,-1.984274,43.317133,0,1,0,0,1,0
2,2,20,20,Donostialdea,Urola Kosta,Andoain,Zarautz,Trabajo habitual,Gestiones personales,Trabajo,...,-2.029870,43.202270,-2.175352,43.284143,0,1,0,0,1,0
3,5,20,20,Goierri,Donostialdea,Beasain,Andoain,Trabajo habitual,Residencia Habitual,Trabajo,...,-2.194873,43.047227,-2.023089,43.211462,0,1,1,0,0,0
4,6,20,20,Donostialdea,Goierri,Andoain,Beasain,Residencia Habitual,Trabajo habitual,Trabajo,...,-2.017355,43.221807,-2.199207,43.044323,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,5080,20,20,Bidasoa,Bidasoa,Irun,Hondarribia,Trabajo habitual,Residencia Habitual,Trabajo,...,-1.767785,43.336861,-1.827935,43.341005,0,1,0,0,1,0
2478,5081,20,20,Bidasoa,Bidasoa,Hondarribia,Irun,Centro estudios,Trabajo habitual,Trabajo,...,-1.818341,43.341179,-1.805404,43.333622,0,1,0,0,1,0
2479,5082,20,20,Bidasoa,Bidasoa,Irun,Hondarribia,Trabajo habitual,Centro estudios,Estudio,...,-1.826807,43.327675,-1.802130,43.372421,0,1,0,0,1,0
2480,5083,20,20,Bidasoa,Bidasoa,Hondarribia,Irun,Residencia Habitual,Trabajo habitual,Trabajo,...,-1.793067,43.369469,-1.849910,43.327468,0,1,0,0,1,0


In [83]:
# # Crea un diccionario, donde los valores son None y las claves son esas cuatro

# networks = dict.fromkeys({
#  "walk",
#  "transit",
#  "train",
#  "drive",
# })

# # Asigna al valor correspondiente en el diccionario networks un objeto de red creado mediante el método from_hdf5() de la clase Network del módulo pdn.network. 
# # Los creados con los codigos anteriores

# for k in networks:
#     print(k)
#     networks[k] = pdn.network.Network.from_hdf5(f'networks/{k}_net.h5')

In [110]:
# Asignar tt a cada modo de transporte para cada trip

# Para cada trip, asigna la distancia mas corta que se puede hacer con Network (walk, drive, transit) con shortest_path_lengths
# Coge lat,long de origen y destino de cada trip. A ese origen y destino le asigna el nodo del Network correspondiente más cercano con get_node_ids
# Le pasa esos nodos del Network correspondiente a shortest_path_lengths, y este calcula la distancia entre esos nodos

for k in networks:
    if k != 'drive':
        trips[f"{k}_tt"] = networks[k].shortest_path_lengths(
            networks[k].get_node_ids(trips.O_long,trips.O_lat),
            networks[k].get_node_ids(trips.D_long,trips.D_lat)
            )

trips["walk_tt"] = trips["walk_tt"] / 80.46 # Pasar de distancia (m) a minutos con un ritmo de 3 mp/h = 80.46 m/min.

trips["drive_tt"] = networks['drive'].shortest_path_lengths(
            networks['drive'].get_node_ids(trips.O_long,trips.O_lat),
            networks['drive'].get_node_ids(trips.D_long,trips.D_lat),
            imp_name='drive_time_s'
            )

trips["drive_tt"] = trips["drive_tt"] / 60 # Para pasar a minutos
trips["bike_tt"] =  networks['drive'].shortest_path_lengths(
            networks['drive'].get_node_ids(trips.O_long,trips.O_lat),
            networks['drive'].get_node_ids(trips.D_long,trips.D_lat),
            imp_name='distance'
            )
trips["bike_tt"] = trips["bike_tt"] * 60 / (1000*13.07)  # 13.07 km/h = 13.07*1000/60 m/min

# Eliminar andando y bike

trips = trips.drop(columns=['walk_tt', 'bike_tt'])

trips['tp_tt'] = trips[['transit_tt', 'train_tt']].min(axis=1)
trips = trips.drop(columns=['transit_tt', 'train_tt'])

trips

c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandana\network.py:334: UserWarning: Unsigned integer: shortest path distance is trying to be calculated                 between the following external unconnected nodes: [(4747441404, 2306456017), (2133500948, 4747441404), (2150637753, 687031601), (553031437, 8318571648), (5320194863, 306109786), (2150637753, 1674826830), (539468419, 2682631024), (2682631024, 1539764105), (9130555771, 904558038), (8235607129, 7908608259), (2150637753, 1801169138), (482975433, 5320435027), (5320435024, 4363663997), (5320409403, 2682631024), (2578257376, 1946748025), (259590790, 619322279), (2943224206, 8235607129), (581995285, 299168656), (6600762366, 2682631024), (2495674088, 4955226153), (531633035, 10308994426), (5320194882, 6600762366), (431064204, 619322279), (8235607129, 311048144), (1630511756, 1497201262), (687031601, 1497201262), (8633045250, 2432248040), (8235607129, 622882945), (2322269990, 595697131)]
  warnings.warn(


,Unnamed: 0,Prov_Ori,Prov_Des,Com_Ori,Com_Des,Mun_Ori,Mun_Des,Act_Ori,Act_Des,Proposito,...,D_long,D_lat,Joven,Mayor,Turismos_1,Turismos_2,Turismos_3,Turismos_4,drive_tt,tp_tt
0,0,20,20,Donostialdea,Goierri,Donostia/San Sebastian,Beasain,Asuntos trabajo,Asuntos trabajo,Trabajo,...,-2.209558,43.043715,0,1,0,0,1,0,28.887217,66.877
1,1,20,20,Debagoiena,Donostialdea,Arrasate/Mondragon,Donostia/San Sebastian,Asuntos trabajo,Asuntos trabajo,Trabajo,...,-1.984274,43.317133,0,1,0,0,1,0,43.306167,69.761
2,2,20,20,Donostialdea,Urola Kosta,Andoain,Zarautz,Trabajo habitual,Gestiones personales,Trabajo,...,-2.175352,43.284143,0,1,0,0,1,0,16.300667,52.984
3,5,20,20,Goierri,Donostialdea,Beasain,Andoain,Trabajo habitual,Residencia Habitual,Trabajo,...,-2.023089,43.211462,0,1,1,0,0,0,18.177500,44.022
4,6,20,20,Donostialdea,Goierri,Andoain,Beasain,Residencia Habitual,Trabajo habitual,Trabajo,...,-2.199207,43.044323,0,1,1,0,0,0,17.583900,43.016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,5080,20,20,Bidasoa,Bidasoa,Irun,Hondarribia,Trabajo habitual,Residencia Habitual,Trabajo,...,-1.827935,43.341005,0,1,0,0,1,0,8.293267,42.036
2478,5081,20,20,Bidasoa,Bidasoa,Hondarribia,Irun,Centro estudios,Trabajo habitual,Trabajo,...,-1.805404,43.333622,0,1,0,0,1,0,3.594117,22.025
2479,5082,20,20,Bidasoa,Bidasoa,Irun,Hondarribia,Trabajo habitual,Centro estudios,Estudio,...,-1.802130,43.372421,0,1,0,0,1,0,11.008133,42.820
2480,5083,20,20,Bidasoa,Bidasoa,Hondarribia,Irun,Residencia Habitual,Trabajo habitual,Trabajo,...,-1.849910,43.327468,0,1,0,0,1,0,13.652300,26.655


# Modify data for MNL

In [111]:
trips = trips.drop(columns=['Unnamed: 0', 'Prov_Ori', 'Prov_Des', 'Com_Ori', 'Com_Des', 'Mun_Ori', 'Mun_Des', 'Act_Ori', 'Act_Des', 'Proposito'])
trips = trips.drop(columns=['Hora_Ini', 'Dur_Tot', 'Recur', 'Bill', 'Park', 'Peaje', 'Provincia', 'Municipio', 'Comarca'])
trips = trips.drop(columns=['Motos', 'Actividad', 'crnt_mot', 'crnt_otr', 'O_long', 'O_lat', 'D_long', 'D_lat'])
trips

,Modo,Per_hog,Sexo,crnt_tur,Joven,Mayor,Turismos_1,Turismos_2,Turismos_3,Turismos_4,drive_tt,tp_tt
0,Coche,2,1,1,0,1,0,0,1,0,28.887217,66.877
1,Coche,2,1,1,0,1,0,0,1,0,43.306167,69.761
2,Coche,2,1,1,0,1,0,0,1,0,16.300667,52.984
3,TP,1,1,1,0,1,1,0,0,0,18.177500,44.022
4,TP,1,1,1,0,1,1,0,0,0,17.583900,43.016
...,...,...,...,...,...,...,...,...,...,...,...,...
2477,Coche,2,1,1,0,1,0,0,1,0,8.293267,42.036
2478,Coche,2,2,1,0,1,0,0,1,0,3.594117,22.025
2479,Coche,2,2,1,0,1,0,0,1,0,11.008133,42.820
2480,Coche,2,2,1,0,1,0,0,1,0,13.652300,26.655


# MNL

In [86]:
# Seleccionar el 20% de los datos de manera aleatoria

# trips = trips.sample(frac=0.05, random_state=42)  # Usamos random_state para obtener la misma muestra en cada ejecución
# trips = trips.reset_index(drop=True)
# trips

In [112]:
# Add availability data to the database

# def convertir_a_nueva_columna(valor):
#     if valor == 1:
#         return 1
#     elif valor == 2:
#         return 0
#     else:
#         return None  # Si deseas tratar otros valores de manera diferente, ajústalos aquí

# trips['drive_av'] = trips['crnt_tur'].apply(convertir_a_nueva_columna)
trips['tp_av'] = 1
trips['drive_av'] = 1


# Elimino las filas que han escogido coche, pero supuestamente no podían porque no tienen carnet

# condicion1 = (trips['crnt_tur'] == 2)
# condicion2 = (trips['Modo'] == 'Coche')

# # Combina las condiciones con un operador lógico "y" (&)
# condiciones_combinadas = condicion1 & condicion2

# # Usa loc para seleccionar las filas que cumplen ambas condiciones y eliminarlas
# trips = trips.loc[~condiciones_combinadas]

# trips = trips.drop(columns='crnt_tur')

# Change choice column
mapear = {'Coche': 0,
          'TP': 1}


# Mapear los valores de la columna 'mode' a números
trips['mode_number'] = trips['Modo'].replace(mapear)
trips = trips.drop(columns='Modo')
trips = trips.reset_index(drop=True)
trips

,Per_hog,Sexo,crnt_tur,Joven,Mayor,Turismos_1,Turismos_2,Turismos_3,Turismos_4,drive_tt,tp_tt,tp_av,drive_av,mode_number
0,2,1,1,0,1,0,0,1,0,28.887217,66.877,1,1,0
1,2,1,1,0,1,0,0,1,0,43.306167,69.761,1,1,0
2,2,1,1,0,1,0,0,1,0,16.300667,52.984,1,1,0
3,1,1,1,0,1,1,0,0,0,18.177500,44.022,1,1,1
4,1,1,1,0,1,1,0,0,0,17.583900,43.016,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,2,1,1,0,1,0,0,1,0,8.293267,42.036,1,1,0
2478,2,2,1,0,1,0,0,1,0,3.594117,22.025,1,1,0
2479,2,2,1,0,1,0,0,1,0,11.008133,42.820,1,1,0
2480,2,2,1,0,1,0,0,1,0,13.652300,26.655,1,1,0


In [113]:
trips = trips.drop(columns=['Per_hog'])
trips['crnt_tur'] = trips['crnt_tur'].replace({2: 0})
trips['Sexo'] = trips['Sexo'].replace({2: 0})
trips

,Sexo,crnt_tur,Joven,Mayor,Turismos_1,Turismos_2,Turismos_3,Turismos_4,drive_tt,tp_tt,tp_av,drive_av,mode_number
0,1,1,0,1,0,0,1,0,28.887217,66.877,1,1,0
1,1,1,0,1,0,0,1,0,43.306167,69.761,1,1,0
2,1,1,0,1,0,0,1,0,16.300667,52.984,1,1,0
3,1,1,0,1,1,0,0,0,18.177500,44.022,1,1,1
4,1,1,0,1,1,0,0,0,17.583900,43.016,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,1,1,0,1,0,0,1,0,8.293267,42.036,1,1,0
2478,0,1,0,1,0,0,1,0,3.594117,22.025,1,1,0
2479,0,1,0,1,0,0,1,0,11.008133,42.820,1,1,0
2480,0,1,0,1,0,0,1,0,13.652300,26.655,1,1,0


In [114]:
# Create the list of individual specific variables
ind_variables = trips.columns.tolist()[:4] # Aqui van todas las filas que no son time, cost, etc.

# Specify the variables that vary across individuals and some or all alternatives
# The keys are the column names that will be used in the long format dataframe.
# The values are dictionaries whose key-value pairs are the alternative id and
# the column name of the corresponding column that encodes that variable for
# the given alternative. Examples below.
alt_varying_variables = {
    u'travel_time': dict([(0, 'drive_tt'),
                                               (1, 'tp_tt')]),
                        #   u'travel_cost': dict([(1, 'drive_tc'),
                        #                        (2, 'transit_tc'),
                        #                        (3, 'train_tc'),
                        #                     #    (4, 'walk_tc'),
                        #                     #    (5, 'bike_tc')
                        #                     ]),
                        #   u'headway': dict([(1, 'TRAIN_HE'),
                        #                     (2, 'SM_HE')]),
                        #   u'seat_configuration': dict([(2, "SM_SEATS")])
                        }

# Specify the availability variables
# Note that the keys of the dictionary are the alternative id's.
# The values are the columns denoting the availability for the
# given mode in the dataset.
availability_variables = {0: 'drive_av',
                          1: 'tp_av'}

##########
# Determine the columns for: alternative ids, the observation ids and the choice
##########
# The 'custom_alt_id' is the name of a column to be created in the long-format data
# It will identify the alternative associated with each row.
custom_alt_id = "mode_id"

# Create a custom id column that ignores the fact that this is a 
# panel/repeated-observations dataset. Note the +1 ensures the id's start at one.
obs_id_column = "custom_id"
trips[obs_id_column] = np.arange(trips.shape[0], dtype=int) + 1                                            

# Create a variable recording the choice column
choice_column = "mode_number"

In [115]:
# Perform the conversion to long-format
long_trips = pl.convert_wide_to_long(trips, 
                                    ind_variables, 
                                    alt_varying_variables, 
                                    availability_variables, 
                                    obs_id_column, 
                                    choice_column,
                                    new_alt_id_name=custom_alt_id)
# Look at the resulting long-format dataframe
long_trips.head(10).T

c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pylogit\choice_tools.py:454: UserWarning: Note, there are 14 variables in wide_data but the inputs ind_vars, alt_specific_vars, and subset_specific_vars only account for 10 variables.
  warnings.warn(msg.format(num_dataframe_vars, data_title) +


,0,1,2,3,4,5,6,7,8,9
custom_id,1.000000,1.000,2.000000,2.000,3.000000,3.000,4.0000,4.000,5.0000,5.000
mode_id,0.000000,1.000,0.000000,1.000,0.000000,1.000,0.0000,1.000,0.0000,1.000
mode_number,1.000000,0.000,1.000000,0.000,1.000000,0.000,0.0000,1.000,0.0000,1.000
Sexo,1.000000,1.000,1.000000,1.000,1.000000,1.000,1.0000,1.000,1.0000,1.000
crnt_tur,1.000000,1.000,1.000000,1.000,1.000000,1.000,1.0000,1.000,1.0000,1.000
Joven,0.000000,0.000,0.000000,0.000,0.000000,0.000,0.0000,0.000,0.0000,0.000
Mayor,1.000000,1.000,1.000000,1.000,1.000000,1.000,1.0000,1.000,1.0000,1.000
travel_time,28.887217,66.877,43.306167,69.761,16.300667,52.984,18.1775,44.022,17.5839,43.016


In [116]:
# NOTE: - Specification and variable names must be ordered dictionaries.
#       - Keys should be variables within the long format dataframe.
#         The sole exception to this is the "intercept" key.
#       - For the specification dictionary, the values should be lists
#         of integers or lists of lists of integers. Within a list, 
#         or within the inner-most list, the integers should be the 
#         alternative ID's of the alternative whose utility specification 
#         the explanatory variable is entering. Lists of lists denote 
#         alternatives that will share a common coefficient for the variable
#         in question.

basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["intercept"] = [1]
basic_names["intercept"] = ['ASC TP']

basic_specification["travel_time"] = [0, 1]
basic_names["travel_time"] = ['Travel Time (Drive)',
                              'Travel Time (TP)']

basic_specification["crnt_tur"] = [1]
basic_names["crnt_tur"] = ['carnet']

basic_specification["Sexo"] = [1]
basic_names["Sexo"] = ['Sexo']

basic_specification["Joven"] = [1]
basic_names["Joven"] = ['Joven']

basic_specification["Mayor"] = [1]
basic_names["Mayor"] = ['Mayor']

# basic_specification["Turismos_1"] = [1]
# basic_names["Turismos_1"] = ['Turismos_1 (TP)']
# basic_specification["Turismos_2"] = [1]
# basic_names["Turismos_2"] = ['Turismos_2 (TP)']
# basic_specification["Turismos_3"] = [1]
# basic_names["Turismos_3"] = ['Turismos_3 (TP)']
# basic_specification["Turismos_4"] = [1]
# basic_names["Turismos_4"] = ['Turismos_4 (TP)']

# basic_specification["Turismos_1"] = [0, 1]
# basic_names["Turismos_1"] = ['Turismos_1 (Drive)', 'Turismos_1 (TP)']
# basic_specification["Turismos_2"] = [0, 1]
# basic_names["Turismos_2"] = ['Turismos_2 (Drive)', 'Turismos_2 (TP)']
# basic_specification["Turismos_3"] = [0, 1]
# basic_names["Turismos_3"] = ['Turismos_3 (Drive)', 'Turismos_3 (TP)']
# basic_specification["Turismos_4"] = [0, 1]
# basic_names["Turismos_4"] = ['Turismos_4 (Drive)', 'Turismos_4 (TP)']

long_trips = long_trips.reset_index(drop=True)
long_trips

,custom_id,mode_id,mode_number,Sexo,crnt_tur,Joven,Mayor,travel_time
0,1,0,1,1,1,0,1,28.887217
1,1,1,0,1,1,0,1,66.877000
2,2,0,1,1,1,0,1,43.306167
3,2,1,0,1,1,0,1,69.761000
4,3,0,1,1,1,0,1,16.300667
...,...,...,...,...,...,...,...,...
4959,2480,1,0,0,1,0,1,42.820000
4960,2481,0,1,0,1,0,1,13.652300
4961,2481,1,0,0,1,0,1,26.655000
4962,2482,0,1,0,1,0,1,5.697650


In [43]:
p=long_trips[long_trips['Turismos_1']==1]
p=p[p['mode_id']==1]
p['mode_number'].sum()

28

In [117]:
# Estimate the multinomial logit model (MNL)
swissmetro_mnl = pl.create_choice_model(data=long_trips,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

# Specify the initial values and method for the optimization.
numCoef=sum([len(basic_specification[s]) for s in basic_specification])
swissmetro_mnl.fit_mle(np.zeros(numCoef), method='BFGS')

# Look at the estimation results
swissmetro_mnl.get_statsmodels_summary()

Log-likelihood at zero: -1,720.3913
Initial Log-likelihood: -1,720.3913


Estimation Time for Point Estimation: 0.04 seconds.
Final log-likelihood: -891.6529


c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_minimize.py:565: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
c:\Users\iazka\AppData\Local\Programs\Python\Python311\Lib\site-packages\pylogit\base_multinomial_cm_v2.py:1259: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.robust_cov)),


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                 mode_number   No. Observations:                2,482
Model:             Multinomial Logit Model   Df Residuals:                    2,475
Method:                                MLE   Df Model:                            7
Date:                     Fri, 13 Oct 2023   Pseudo R-squ.:                   0.482
Time:                             09:34:03   Pseudo R-bar-squ.:               0.478
AIC:                             1,797.306   Log-Likelihood:               -891.653
BIC:                             1,838.024   LL-Null:                    -1,720.391
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
ASC TP                  0.9454   1.14e+06   8.28e-07      1.000   -2.24e+06    2.24e+06
Travel Time (Drive) -5.529e-06   1.55e-05     -0.356      0.722    -3.6e-05    2.49e-05
Travel Time (TP)     5.292e-07   1.88e-07      2.815      0.005    1.61e-07    8.98e-07
carnet                 -2.8070      0.183    -15.351      0.000      -3.165      -2.449
Sexo                   -1.3126      0.128    -10.234      0.000      -1.564      -1.061
Joven                   0.4509   1.14e+06   3.95e-07      1.000   -2.24e+06    2.24e+06
Mayor                   0.4945   1.14e+06   4.33e-07      1.000   -2.24e+06    2.24e+06
=======================================================================================
"""

In [28]:
# Look at the parameter estimation results, and round the results for easy viewing
np.round(swissmetro_mnl.summary, 5)

,parameters,std_err,t_stats,p_values,robust_std_err,robust_t_stats,robust_p_values
ASC TP,-16.85995,1.182006e+06,-0.00001,0.99999,NaN,NaN,NaN
Travel Time (Drive),-0.00001,2.000000e-05,-0.39504,0.69282,0.00002,-0.35658,0.72141
Travel Time (TP),0.00000,0.000000e+00,2.19796,0.02795,0.00000,2.11874,0.03411
carnet,-2.03535,1.973100e-01,-10.31526,0.00000,0.20931,-9.72399,0.00000
Sexo,-1.48529,1.380400e-01,-10.75985,0.00000,0.13718,-10.82721,0.00000
Joven,22.60339,9.130948e+04,0.00025,0.99980,NaN,NaN,NaN
Mayor,22.12843,9.130948e+04,0.00024,0.99981,NaN,NaN,NaN
Turismos_1 (TP),-2.09986,1.181904e+06,-0.00000,1.00000,NaN,NaN,NaN
Turismos_2 (TP),-4.27510,1.181904e+06,-0.00000,1.00000,NaN,NaN,NaN
Turismos_3 (TP),-5.33792,1.181904e+06,-0.00000,1.00000,NaN,NaN,NaN


In [60]:
# Look at other all results at the same time
swissmetro_mnl.print_summaries()



Number of Parameters                                                     14
Number of Observations                                                 7238
Null Log-Likelihood                                            -11649.11161
Fitted Log-Likelihood                                          -8949.825401
Rho-Squared                                                        0.231716
Rho-Bar-Squared                                                    0.230514
Estimation Message        Desired error not necessarily achieved due to ...
dtype: object
                         parameters       std_err       t_stats  \
ASC Transit           -7.100745e-01  3.660030e-02 -1.940079e+01   
ASC Train             -9.500617e-01  3.949406e-02 -2.405582e+01   
ASC Walk              -2.271043e+00  6.854141e-02 -3.313388e+01   
ASC Bike              -1.071218e-01  1.432640e-01 -7.477227e-01   
Travel Time (Drive)    1.186841e-05  4.816938e-06  2.463890e+00   
Travel Time (Transit)  2.171157e-07  1.233619e-07 

In [27]:
# Look at the general and goodness of fit statistics
swissmetro_mnl.fit_summary

Number of Parameters                                                     11
Number of Observations                                                 7682
Null Log-Likelihood                                           -12363.702043
Fitted Log-Likelihood                                          -9973.740656
Rho-Squared                                                        0.193305
Rho-Bar-Squared                                                    0.192415
Estimation Message        Desired error not necessarily achieved due to ...
dtype: object